In [ ]:
# import modules 
#%matplotlib notebook
import json
import requests
#from config import api_key
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import csv
%matplotlib notebook
import matplotlib
import matplotlib.dates as mdates
import matplotlib.ticker as mtick
import datetime
from time import strptime
import re

In [ ]:
# Read In 
# Nicks Exchange Rate Tradition Currency CSV File Input 
file = "Data/Exchange_Rate_Report1.csv"
clean_foreign_ex = pd.read_csv(file,header=2)
clean_foreign_ex = clean_foreign_ex[:-34]
clean_foreign_ex

In [ ]:
# Read In 
# Daves input cryptocurrency data 

cryptodatafile = 'Data/merged_crypto_data.csv'
RawCrypto_df = pd.read_csv(cryptodatafile)
RawCrypto_df.head()

In [ ]:
# Clean up 
# Foreign Exchange Data Cleanup 
# Set columns for foreign exchange 
clean_foreign_ex.columns = ['Date','Australian Dollar (AUD)','Chinese Yuan (CNY)','Euro (EUR)','Japanese Yen (JPY)','Swiss Franc (CHF)','U.K. Pound (GBP)','U.S. Dollar (USD)']

#reduce dataframe to currencies we need.  AUD, Pound, EURO, Yuan, Yen, Swiss. Set AUD to Float dtype. 
forex_clean = clean_foreign_ex[["Date", "Australian Dollar (AUD)", "Euro (EUR)", "U.K. Pound (GBP)", "U.S. Dollar (USD)"]]
forex_clean['Australian Dollar (AUD)'] = forex_clean['Australian Dollar (AUD)'].astype(float)
forex_clean.head()

In [ ]:
# Clean Up 
# Nicks's Traditional Currency Cleanup
# Read in String date format, convert to Datestamp obejct and read in as dates 
def monthToNum(shortMonth):
    return {
            'jan' : 1,
            'feb' : 2,
            'mar' : 3,
            'apr' : 4,
            'may' : 5,
            'jun' : 6,
            'jul' : 7,
            'aug' : 8,
            'sep' : 9, 
            'oct' : 10,
            'nov' : 11,
            'dec' : 12
    }[shortMonth]
splitdate = pd.DataFrame()
splitdate[['day','month','year']] = forex_clean['Date'].str.split("\s|-", 2, expand=True)
splitdate['month'] = splitdate['month'].str.lower()
splitdate['month'] = splitdate['month'].apply(monthToNum)

# Insert Datestamp YYYMMDD Column for sorting 
forex_clean.insert(1,'Datestamp',pd.to_datetime(splitdate[['year','month','day']]))

# Fill Empty Cells 
for_ex_clean = forex_clean.fillna(method='ffill')
for_ex_clean

forex_clean

In [ ]:
# Clean Up 
# Dave's Crypto Currency Cleanup

# Function to convert "Date" to usable / sortable format 
def monthToNum(shortMonth):
    return {
            'jan' : 1,
            'feb' : 2,
            'mar' : 3,
            'apr' : 4,
            'may' : 5,
            'jun' : 6,
            'jul' : 7,
            'aug' : 8,
            'sep' : 9, 
            'oct' : 10,
            'nov' : 11,
            'dec' : 12
    }[shortMonth]
splitdate = pd.DataFrame()
splitdate[['month','day','year']] = RawCrypto_df['Date'].str.split("\s|,", 2, expand=True)
splitdate['month'] = splitdate['month'].str.lower()
splitdate['month'] = splitdate['month'].apply(monthToNum)

# Insert Datestamp YYYMMDD Column for sorting 
RawCrypto_df.insert(1,'Datestamp',pd.to_datetime(splitdate[['year','month','day']]))
RawCrypto_df.dtypes

# Recast Coin names as Title-case 
RawCrypto_df['Coin'] = RawCrypto_df['Coin'].str.title()

# Write RawCrypto df to file
RawCrypto_df.to_csv('Data/RawCrypto_df.csv')
RawCrypto_df

In [ ]:
# Exploration  - Nick 
# Nick Traditional Currency Groupby Tables 
groupby_mean_df = forex_clean.groupby(pd.Grouper(key='Datestamp',freq='1M')).mean()
forex_groupby_std_df = forex_clean.groupby(pd.Grouper(key='Datestamp',freq='1M')).std()
forex_groupby_var_df = forex_clean.groupby(pd.Grouper(key='Datestamp',freq='1M')).var()





In [ ]:
# Exploration - David 
# Dave's Coinlist and Colourlist 

coinlist = ['Bitcoin','Ethereum','Litecoin','Iota','Monero','Dash']

colourdict = {'Bitcoin':'dodgerblue',
'Ethereum':'orangered',
'Litecoin':'orange',
'Iota':'greenyellow',
'Monero':'forestgreen',
'Dash':'darkturquoise'}

In [ ]:
# Exploration - Nick
# Nick's Traditional Currency & Bitcoin Dataframe - create dataframes for combining later 
cryptodatafile = 'Data/RawCrypto_df.csv'
RawCrypto_df = pd.read_csv(cryptodatafile)

RawCrypto_df['Datestamp'] = pd.to_datetime(RawCrypto_df['Datestamp'], yearfirst= True)

ethereum_df = RawCrypto_df.loc[RawCrypto_df['Coin'] == 'Ethereum'].sort_values(by='Datestamp')
bitcoin_df = RawCrypto_df.loc[RawCrypto_df['Coin'] == 'Bitcoin'].sort_values(by='Datestamp')
litecoin_df = RawCrypto_df.loc[RawCrypto_df['Coin'] == 'Litecoin'].sort_values(by='Datestamp')

bitcoin_std_groupbymonth_df = bitcoin_df.groupby(pd.Grouper(key='Datestamp',freq='1M')).std()
ethereum_std_groupbymonth_df = ethereum_df.groupby(pd.Grouper(key='Datestamp',freq='1M')).std()
litecoin_std_groupbymonth_df = litecoin_df.groupby(pd.Grouper(key='Datestamp',freq='1M')).std()

bitcoin_var_groupbymonth_df = bitcoin_df.groupby(pd.Grouper(key='Datestamp',freq='1M')).var()
ethereum_var_groupbymonth_df = ethereum_df.groupby(pd.Grouper(key='Datestamp',freq='1M')).var()
litecoin_var_groupbymonth_df = litecoin_df.groupby(pd.Grouper(key='Datestamp',freq='1M')).var()

bitcoin_mean_groupbymonth_df = bitcoin_df.groupby(pd.Grouper(key='Datestamp',freq='1M')).mean()
ethereum_mean_groupbymonth_df = ethereum_df.groupby(pd.Grouper(key='Datestamp',freq='1M')).mean()
litecoin_mean_groupbymonth_df = litecoin_df.groupby(pd.Grouper(key='Datestamp',freq='1M')).mean()


bitcoin_mean_groupbymonth_df




In [ ]:
# Exploration - Nick
# Create combined STD DEV dataframe with Trad Currency and Crypto Currency 
combined_std_df = pd.DataFrame()

combined_std_df['Australian Dollar (AUD)'] = forex_groupby_std_df['Australian Dollar (AUD)']
combined_std_df['Euro (EUR)'] = forex_groupby_std_df['Euro (EUR)']
combined_std_df['U.K. Pound (GBP)'] = forex_groupby_std_df['U.K. Pound (GBP)']

combined_std_df['Bitcoin'] = bitcoin_std_groupbymonth_df['Close']
combined_std_df['Ethereum'] = ethereum_std_groupbymonth_df['Close']
combined_std_df['Litecoin'] = litecoin_std_groupbymonth_df['Close']

In [ ]:
# Exploration - Nick
# Calculate Mean for AUD, CNY, EUR, JPY, CHF, GBP

AUD_mean = forex_clean['Australian Dollar (AUD)'].mean()
EURO_mean = forex_clean['Euro (EUR)'].mean()
GBP_mean = forex_clean['U.K. Pound (GBP)'].mean()
USD_mean = forex_clean['U.S. Dollar (USD)'].mean()
print(f"The mean for AUD is {AUD_mean}")
print(f"The mean for EURO is {EURO_mean}")
print(f"The mean for GBP is {GBP_mean}")
print(f"The mean for USD is {USD_mean}")

In [ ]:
# Push the remade DataFrame to a new CSV file
for_ex_clean.to_csv("Data/for_ex_clean.csv",
                  encoding="utf-8", index=False, header=True)

In [ ]:
# Dave - Exploration
# Loop and create coin-based dataframes for each cryptocurrency 

for coin in coinlist:
    globals()[f"{coin}_df"] = RawCrypto_df.loc[RawCrypto_df['Coin'] == coin].sort_values(by='Datestamp')
    exec(f"{coin}_groupbymonth_df = {coin}_df.groupby(pd.Grouper(key='Datestamp',freq='1M'))")

Bitcoin_groupbymonth_df.mean()



In [ ]:
# Nick -Exploration and Analysis
# get max and min values of traditional currencies 

#MAX and MIN
aud_max = for_ex_clean['Australian Dollar (AUD)'].max()
aud_min = for_ex_clean['Australian Dollar (AUD)'].min()

euro_max = for_ex_clean['Euro (EUR)'].max()
euro_min = for_ex_clean['Euro (EUR)'].min()

gbp_max = for_ex_clean['U.K. Pound (GBP)'].max()
gbp_min = for_ex_clean['U.K. Pound (GBP)'].min()

print(f"The max value of USD vs AUD is {aud_max} and the min value of USD vs AUD is {aud_min}.")
print(f"The max value of USD vs EURO is {euro_max} and the min value of USD vs EURO is {euro_min}.")
print(f"The max value of USD vs GBP is {gbp_max} and the min value of USD vs GBP is {gbp_min}.")
print(f"---------------------------------------------------------------------------------------------------")


#Standard Deviation
aud_std = for_ex_clean['Australian Dollar (AUD)'].std()
euro_std = for_ex_clean['Euro (EUR)'].std()
gbp_std = for_ex_clean['U.K. Pound (GBP)'].std()

print(f"The Standard Deviation for the AUD is {aud_std}.")
print(f"The Standard Deviation for the Euro is {euro_std}.")
print(f"The Standard Deviation for the GBP is {gbp_std}.")
print(f"---------------------------------------------------------------------------------------------------")

#Mean
aud_mean = for_ex_clean['Australian Dollar (AUD)'].mean()
euro_mean = for_ex_clean['Euro (EUR)'].mean()
gbp_mean = for_ex_clean['U.K. Pound (GBP)'].mean()

print(f"The Mean for the AUD is {aud_mean}.")
print(f"The Mean for the Euro is {euro_mean}.")
print(f"The Mean for the GBP is {gbp_mean}.")
print(f"---------------------------------------------------------------------------------------------------")

#Mode
aud_mode = for_ex_clean['Australian Dollar (AUD)'].mode()
euro_mode = for_ex_clean['Euro (EUR)'].mode()
gbp_mode = for_ex_clean['U.K. Pound (GBP)'].mode()

print(f"The Mode for the AUD is {aud_mode}.")
print(f"The Mode for the Euro is {euro_mode}.")
print(f"The Mode for the GBP is {gbp_mode}.")
print(f"---------------------------------------------------------------------------------------------------")


#Variance 
aud_var = for_ex_clean['Australian Dollar (AUD)'].var()
euro_var = for_ex_clean['Euro (EUR)'].var()
gbp_var = for_ex_clean['U.K. Pound (GBP)'].var()

print(f"The Variance for the AUD is {aud_var}.")
print(f"The Variance for the Euro is {euro_var}.")
print(f"The Variance for the GBP is {gbp_var}.")
print(f"---------------------------------------------------------------------------------------------------")


In [ ]:
# Nick - Exploration 
# AUD Currency Fluctuations
plt.plot(groupby_mean_df['Australian Dollar (AUD)'])
plt.plot(forex_groupby_std_df['Australian Dollar (AUD)']) 
plt.xlabel("Date",size=16)
plt.ylabel("USD/AUD",size=16)
plt.title("AUD Currency Fluctuations", size=18)
plt.gca().legend(('Australian Dollar (AUD)','Australian Dollar (AUD) std'))
plt.tight_layout()
plt.savefig("Output/AUD Currency Fluctuations.png,dpi=300")

In [ ]:
# Nick - Exploration 
#Euro Currency Fluctuations
plt.plot(groupby_mean_df['Euro (EUR)'])
plt.plot(forex_groupby_std_df['Euro (EUR)']) 
plt.xlabel("Date",size=16)
plt.ylabel("USD/EURO",size=16)
plt.title("Euro Currency Fluctuations", size=18)
plt.gca().legend(('Euro (EUR)','Euro (EUR) std'))
plt.tight_layout()
plt.savefig("Output/Euro Currency Fluctuations.png,dpi=300")

In [ ]:
# Nick - Exploration 
#GBP Currency Fluctuations
plt.plot(groupby_mean_df['U.K. Pound (GBP)'])
plt.plot(forex_groupby_std_df['U.K. Pound (GBP)']) 
plt.xlabel("Date",size=16)
plt.ylabel("USD/GBP",size=16)
plt.title("GBP Currency Fluctuations", size=18)
plt.gca().legend(('U.K. Pound (GBP)','U.K. Pound (GBP) std'))
plt.tight_layout()
plt.savefig("Output/GBP Currency Fluctuations.png,dpi=300")

In [ ]:
# Nick - Exploration 
#Bitcoin Fluctuations 
plt.plot(combined_mean_df['Bitcoin'])
plt.plot(combined_std_df['Bitcoin']) 
plt.xlabel("Date",size=16)
plt.ylabel("USD/Bitcoin",size=16)
plt.title("Bitcoin Currency Fluctuations", size=18)
plt.gca().legend(('Bitcoin','Bitcoin std'))
plt.tight_layout()
plt.savefig("Output/Bitcoin Currency Fluctuations.png,dpi=300")

In [ ]:
# Nick - Exploration 
#Ethereum Fluctuations
plt.plot(combined_mean_df['Ethereum'])
plt.plot(combined_std_df['Ethereum']) 
plt.xlabel("Date",size=16)
plt.ylabel("USD/Ethereum",size=16)
plt.title("Ethereum Currency Fluctuations", size=18)
plt.gca().legend(('Ethereum','Ethereum std'))
plt.tight_layout()
plt.savefig("Output/Ethereum Currency Fluctuations.png,dpi=300")

In [ ]:
# Nick - Exploration 
#Litecoin Fluctuations
plt.plot(combined_mean_df['Litecoin'])
plt.plot(combined_std_df['Litecoin']) 
plt.xlabel("Date",size=16)
plt.ylabel("USD/Litecoin",size=16)
plt.title("Litecoin Currency Fluctuations", size=18)
plt.gca().legend(('Litecoin','Litecoin std'))
plt.tight_layout()
plt.savefig("Output/Litecoin Currency Fluctuations.png,dpi=300")

In [ ]:
# Nick - Exploration 
#Combined Crypto and Traditional Currencies Fluctuation in STD
plt.plot(combined_std_df)
plt.legend(loc = 'upper left')
plt.ylim(-10, 100)
plt.xlabel("Date",size=16)
plt.ylabel("USD/Currency",size=16)
plt.title("Traditional and Crypto Fluctuations(Std)", size=18)
plt.tight_layout()
plt.savefig("Output/Traditional Vs Crypto-Currencies Fluctuations.png,dpi=300")

In [ ]:
# Nick - Graphing 
# Graph STD DEV of crypto fluctuations 
plt.plot(combined_std_df['Bitcoin'])
plt.plot(combined_std_df['Ethereum'])
plt.plot(combined_std_df['Litecoin'])
plt.legend(loc = 'upper left')
plt.ylim(0, 100)
plt.xlabel("Date",size=16)
plt.ylabel("USD/Currency",size=16)
plt.title("Cryptocurrency Fluctuations (Std)", size=18)
plt.gca().legend(('Bitcoin','Ethereum', 'Litecoin'))
plt.tight_layout()
plt.savefig("Output/Crypto-Currencies Fluctuations.png,dpi=300")

In [ ]:
# Nick - Graphing and Analysis 
#Traditional Cryptocurrency Fluctuations in STD
plt.plot(forex_groupby_std_df['U.K. Pound (GBP)']) 
plt.plot(forex_groupby_std_df['Euro (EUR)']) 
plt.plot(forex_groupby_std_df['Australian Dollar (AUD)'])
plt.ylim(0, 0.2)
plt.xlabel("Date",size=16)
plt.ylabel("USD/Currency",size=16)
plt.title("Traditional Currencies Fluctuations (Std)", size=18)
plt.gca().legend(('U.K. Pound (GBP)std','Euro (EUR)std', 'Australian Dollar (AUD)std'))
plt.tight_layout()
plt.savefig("Output/Traditional Currency Fluctuations.png,dpi=300")

In [ ]:
# David - Anaylsis and Graphing
# Plot Cryptocurrency Closing Price over time  
# xticklist = [2014,2015,2016,2017,2018,2019,2020,2021,2022]
for coin in coinlist:
    plt.figure()
    exec(f"ax = {coin}_df.plot('Datestamp','Close',label='{coin}',color=colourdict['{coin}'])")
    exec(f"plt.title('{coin} Closing Price Over Time')")
    plt.setp(ax.get_xticklabels(), rotation = 30)
    ax.set_xlim(pd.Timestamp('2014-01-01'), pd.Timestamp('2021-03-28'))
    plt.tight_layout()
    fmt = '${x:,.0f}'
    tick = mtick.StrMethodFormatter(fmt)
    ax.yaxis.set_major_formatter(tick) 
    exec(f"ax.yaxis.set_tick_params(which='major', labelcolor='black')")
    exec(f"plt.savefig('Output/{coin}_Closing_Price_Over_Time.png',dpi=300)")

plt.show()

In [ ]:
# David - Anaylsis and Graphing
# Plot Cryptocurrency Average Monthly Price 
for coin in coinlist:
    plt.figure()
    exec(f"plt.plot({coin}_groupbymonth_df[['Close']].mean(), color=colourdict['{coin}'])")
    exec(f"plt.title('Mean of {coin} Monthly Close prices')")
    plt.xlim([datetime.date(2014,1,1), datetime.date(2021,4,28)])
    plt.tight_layout()
    plt.ylabel('Price per coin in $ USD')
    exec(f"plt.savefig('Output/{coin}_Average Monthly Price.png',dpi=300)")

In [ ]:
# David - Anaylsis and Graphing
# Plot Cryptocurrency Standard Deviation of Monthly Prices
for coin in coinlist:
    plt.figure()
    exec(f"plt.plot({coin}_groupbymonth_df[['High']].std(),color=colourdict['{coin}'])")
    exec(f"plt.title('Standard Deviation of {coin} Monthly High prices')")
    plt.xlim([datetime.date(2014,1,1), datetime.date(2021,4,28)])
    plt.ylabel('Standard Deviation of Prices')
    plt.xlabel("Timepoint")
    plt.tight_layout()
    exec(f"plt.savefig('Output/{coin}_Standard Deviation of Monthly High Prices.png',dpi=300)")

In [ ]:
# David - Anaylsis and Graphing
# Plot Cryptocurrency Variance of Monthly Prices
for coin in coinlist:
    plt.figure()
    exec(f"plt.plot({coin}_groupbymonth_df[['High']].var(),color=colourdict['{coin}'])")
    exec(f"plt.title('Variance of {coin} Monthly High/Low prices')")  
    plt.xlim([datetime.date(2014,1,1), datetime.date(2021,4,28)]) 
    plt.tight_layout()
    exec(f"plt.savefig('Output/{coin} Variance of Monthly High Price.png',dpi=300)")

In [ ]:
# David - Anaylsis and Graphing
# Plot Cryptocurrency Min vs Max Prices 

from matplotlib.ticker import FuncFormatter

for coin in coinlist:
    # Create new Delta Dataframe
    exec(f"{coin}_delta = {coin}_groupbymonth_df['High'].max()")
    exec(f"{coin}_delta = pd.DataFrame({coin}_delta)")
    exec(f"{coin}_delta['Low'] = {coin}_groupbymonth_df['Low'].min()")
    exec(f"{coin}_delta['HighLowDelta'] = {coin}_delta['High'] - {coin}_delta['Low']")
    exec(f"{coin}_delta['Close'] = {coin}_groupbymonth_df['Close'].max()")
    exec(f"{coin}_delta['CloseDiff'] = {coin}_delta['Close'].diff()")
    exec(f"{coin}_delta['PctChange'] = ({coin}_delta['CloseDiff']) / ({coin}_delta['Close'] - {coin}_delta['CloseDiff']) * 100 ")    
    # Plot Graphs
    plt.figure()
    fig, ax = plt.subplots()
    exec(f"plt.plot({coin}_delta['High'], color=colourdict['{coin}'])")
    exec(f"plt.plot({coin}_delta['Low'], color=colourdict['{coin}'])")
    exec(f"{coin}_delta['x1'] = {coin}_delta.index")
    exec(f"plt.fill_between(x={coin}_delta['x1'], y1={coin}_delta['Low'], y2={coin}_delta['High'], color=colourdict['{coin}'])")
    exec(f"plt.suptitle('{coin} Monthly Highest vs Lowest price')")
    exec(f"plt.title('Thicker Line means Higher Volatility')")
    fmt = '${x:,.0f}'
    tick = mtick.StrMethodFormatter(fmt)
    ax.yaxis.set_major_formatter(tick) 
    plt.xlim([datetime.date(2014,1,1), datetime.date(2021,4,28)])
    plt.tight_layout()
    exec(f"plt.savefig('Output/{coin} Monthly Highest vs Lowest Price.png',dpi=300)")


In [ ]:
# Analysis and Graphing - David
# Create % Change dataframe and plot results 

PctChange_df = pd.DataFrame()
for coin in coinlist:
    # copy PctChange colum of various DFs into CoinPctChange Column in One DF
    exec(f"PctChange_df['{coin}PctChange'] = {coin}_delta['PctChange']")

# calculate the mean % change of all coins in table 
PctChange_df['mean'] = PctChange_df.iloc[:, 0:5].mean(axis=1)

# plot results 
for coin in reversed(coinlist):
    exec(f"plt.plot({coin}_delta[['PctChange']],color=colourdict['{coin}'], linewidth =1.2, label='{coin}' )")
    plt.xlabel('Timepoint')
    plt.ylabel('% Increase / Decrease')
    plt.xlim([datetime.date(2014,1,1), datetime.date(2021,4,28)])
    plt.ylim(-60,350)
    plt.axhline(y=0, color = 'black', linewidth = .5)
    plt.tight_layout()

# plot results continued and save 
exec(f"plt.title('% Change of Monthly Mean prices')")
plt.plot(PctChange_df['mean'], linewidth=3.5, label='Mean of % Change')
plt.legend(loc="upper left")
exec(f"plt.savefig('Output/All Coins - Percent Change in Monthly Prices.png',dpi=300)")

In [ ]:
# David - Anaylsis and Graphing
# Calculate Log, Square and Square Root of rolling average 
window = 21
dpy = 252
ann_factor = dpy / window

for coin in reversed(coinlist):
    exec(f"{coin}Log_df = pd.DataFrame()")
    exec(f"{coin}Log_df['Datestamp'] = {coin}_df['Datestamp']")
    exec(f"{coin}Log_df['Close'] = {coin}_df['Close']")
    exec(f"{coin}Log_df = {coin}Log_df.set_index(['Datestamp'])")
    exec(f"{coin}Log_df['Close'] = np.log({coin}Log_df['Close'])")

    exec(f"{coin}Log_df['LogRtn'] = {coin}Log_df[['Close']].diff()")
    exec(f"{coin}Log_df['real_var'] = np.square({coin}Log_df['LogRtn']).rolling(window).sum() * ann_factor")
    exec(f"{coin}Log_df['real_vol'] = np.sqrt({coin}Log_df['real_var'])")

    exec(f"plt.plot({coin}Log_df['real_vol'], color=colourdict['{coin}'])")
plt.show()

In [ ]:
# David - Analysis and Graphing 
# Calculate Volume of Trades Each month

for coin in coinlist:
    exec(f"{coin}_groupbymonth_df['Volume'].sum().plot(label='{coin}', color=colourdict['{coin}'])")
    plt.title('Volume of Trades Each Month - Trillions')
    plt.ylabel('Number of Trades in Trillions')
    plt.xlim([datetime.date(2016,1,1), datetime.date(2021,3,14)])
    # plt.gca.legend((coinlist))
    plt.legend(loc="upper left")
    plt.tight_layout()
    exec(f"plt.savefig('Output/All Coins - Volume of Trades Per Month.png',dpi=300)")
plt.show()

In [ ]:
# David - Analysis and Graphing
# Calculate Standard Deviation of Trade Volumes 

for coin in coinlist:
    exec(f"{coin}_groupbymonth_df['Volume'].std().plot(label='{coin}', color=colourdict['{coin}'])")
    plt.title('Std Dev of Trades Each Month')
    plt.ylabel('Std Dev of Trades')
    plt.xlim([datetime.date(2016,1,1), datetime.date(2021,3,14)])
    # plt.gca.legend((coinlist))
    plt.legend(loc="upper left")
    plt.tight_layout()
    exec(f"plt.savefig('Output/All Coins - Standard Deviation of Trades Per Month.png',dpi=300)")
plt.show()

In [67]:
# David - Analysis and Graphing
# Calculate Log return of Standard Deviation of Coin Prices 

for coin in reversed(coinlist):
    plt.figure
    exec(f"plt.plot(np.log({coin}_groupbymonth_df[['High']].std()),label={'coin'}, color=colourdict['{coin}'])")
    plt.title('Log of Std Deviation of Coin Prices')
    ax.set_xlim(pd.Timestamp('2014-01-01'), pd.Timestamp('2021-03-28'))  
    plt.legend(loc="upper left")
    exec(f"plt.savefig('Output/All Coins - Log Return of Standard Deviation of Coin Prices.png',dpi=300)")
plt.show()

<IPython.core.display.Javascript object>

<string>:1: RuntimeWarning: divide by zero encountered in log


In [ ]:
# David - Analysis and Graphing
# Calculate Log return of Monthly highest coin prices 

for coin in coinlist:
    plt.figure
    exec(f"plt.plot(np.log({coin}_groupbymonth_df[['High']].max()))")
    plt.title('Log of Monthly Max Coin Prices')
    ax.set_xlim(pd.Timestamp('2014-01-01'), pd.Timestamp('2021-03-28'))  
